Demonstration of GPR for infant mortality data in Miller (2015) with proposed accountability measure in LLM (2020)

Miller, Michael K. "Electoral authoritarianism and human development." Comparative Political Studies 48.12 (2015): 1526-1562.

LÜHRMANN, A., MARQUARDT, K., & MECHKOVA, V. (2020). Constraining Governments: New Indices of Vertical, Horizontal, and Diagonal Accountability. American Political Science Review, 114(3), 811-820.

In [1]:
# load packages
import torch
import numpy as np
import pandas as pd
from scipy.stats import norm
import gpytorch
from typing import Optional, Tuple
from matplotlib import pyplot as plt
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.means import LinearMean
from gpytorch.kernels import ScaleKernel, RBFKernel
from gpytorch.variational import CholeskyVariationalDistribution
from gpytorch.variational import VariationalStrategy

Implement constant mean module and mask mean module

In [2]:
class ConstantVectorMean(gpytorch.means.mean.Mean):
    def __init__(self, d=1, prior=None, batch_shape=torch.Size(), **kwargs):
        super().__init__()
        self.batch_shape = batch_shape
        self.register_parameter(name="constantvector",\
                 parameter=torch.nn.Parameter(torch.zeros(*batch_shape, d)))
        if prior is not None:
            self.register_prior("mean_prior", prior, "constantvector")

    def forward(self, input):
        return self.constantvector[input.int().reshape((-1,)).tolist()]
    
class MaskMean(gpytorch.means.mean.Mean):
    def __init__(
        self,
        base_mean: gpytorch.means.mean.Mean,
        active_dims: Optional[Tuple[int, ...]] = None,
        **kwargs,
    ):
        super().__init__()
        if active_dims is not None and not torch.is_tensor(active_dims):
            active_dims = torch.tensor(active_dims, dtype=torch.long)
        self.active_dims = active_dims
        self.base_mean = base_mean
    
    def forward(self, x, **params):
        return self.base_mean.forward(x.index_select(-1, self.active_dims), **params)

load data

In [6]:
# read data and drop NA
data = pd.read_csv("accountability_data_regressions.csv", index_col=[0])
# data = data[~data.infant.isna()]
data = data[["infant", "Accountability", "aid", "loggdp", "gdp_grow",\
            "resourcesdep_hm", "gini2", "lnpop", "urban_cow", "country_name", \
            "violence_domestic", "communist", "rx_infant", "v2x_corr", "Vertical", "Horizontal"]]

data = data.dropna()

countries = sorted(data.country_name.unique())
data["year"] = data.index.values
years = data.index.unique()
n = len(countries)
m = len(years)

# build data
country_dict = dict(zip(countries, range(n)))
year_dict = dict(zip(years, range(m)))

# x is:
# 0: year number
# 1: country id
# 2: Accountability (regressor of interests)
# 3: aid
# 4: loggdp
# 5: gdp_grow
# 6: resourcesdep_hm
# 7: gini2
# 8: lnpop
# 9: urban_cow
# 10: rx_infant
# 11: v2x_corr 
# 12: violence_domestic
# 13: communist
# 14: Vertical
# 15: Horizontal

x = torch.zeros(data.shape[0], 16)
x[:,0] = torch.as_tensor(list(map(year_dict.get, data.index)))
x[:,1] = torch.as_tensor(list(map(country_dict.get, data.country_name)))
x[:,2] = torch.as_tensor(data.Accountability.to_numpy())
x[:,3] = torch.as_tensor(data.aid.to_numpy())
x[:,4] = torch.as_tensor(data.loggdp.to_numpy())
x[:,5] = torch.as_tensor(data.gdp_grow.to_numpy())
x[:,6] = torch.as_tensor(data.resourcesdep_hm.to_numpy())
x[:,7] = torch.as_tensor(data.gini2.to_numpy())
x[:,8] = torch.as_tensor(data.lnpop.to_numpy())
x[:,9] = torch.as_tensor(data.urban_cow.to_numpy())
x[:,10] = torch.as_tensor(data.rx_infant.to_numpy())
x[:,11] = torch.as_tensor(data.v2x_corr.to_numpy())
x[:,12] = torch.as_tensor(data.violence_domestic.to_numpy())
x[:,13] = torch.as_tensor(data.communist.to_numpy())
x[:,14] = torch.as_tensor(data.Vertical.to_numpy())
x[:,15] = torch.as_tensor(data.Horizontal.to_numpy())
y = torch.as_tensor(data.infant.to_numpy()).float()

train_x, train_y = x.float(), y.float()


Build GPR model with unit trends

In [7]:
# model specification: gp model with unit trends
# infant ~ Accountability + aid + loggdp + gdp_grow +
#    resourcesdep_hm + gini2 + lnpop + urban_cow + 
#    violence_domestic + communist + rx_infant + v2x_corr + u_i(t)
# u_i(t) ~ GP(b_i, K)
# K = K_x + K_t*K_{spatial}

class GPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, n_country):
        # variational_distribution = CholeskyVariationalDistribution(train_x.size()[0])
        # variational_strategy = VariationalStrategy(
        #     self, train_x, variational_distribution, learn_inducing_locations=False
        # )
        # super(GPModel, self).__init__(variational_strategy)
        super(GPModel, self).__init__(train_x, train_y, likelihood)
        # unit mean by country
        self.unit_mean_module = MaskMean(active_dims=1, base_mean=ConstantVectorMean(d=n_country))
        # year kernel * country kernel
        self.unit_covar_module = ScaleKernel(RBFKernel(active_dims=0)*RBFKernel(active_dims=[14,15]))
        # adopt a product kernel of continuous inputs and categorical inputs
        self.x_covar_module = ScaleKernel(RBFKernel(active_dims=[2,3,4,5,6,7,8,9,10,11], ard_num_dims=10))
        # linear mean for continuous covariates
        self.x_mean_module = MaskMean(active_dims=[2,3,4,5,6,7,8,9,10,11], base_mean=LinearMean(input_size=10, bias=False))
        # dummy mean for categorical covariates
        self.categorical_mean_module = torch.nn.modules.ModuleList([MaskMean(active_dims=12, base_mean=ConstantVectorMean(d=11)),\
                                                                    MaskMean(active_dims=13, base_mean=ConstantVectorMean(d=2))])

    def forward(self, x):
        mean_x = self.unit_mean_module(x) + self.x_mean_module(x)
        for i in range(len(self.categorical_mean_module)):
            mean_x += self.categorical_mean_module[i](x)
        # K = K_country * K_year + K_continuous 
        covar_x = self.x_covar_module(x) + self.unit_covar_module(x)
        
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

build a 2FE model

In [8]:
import statsmodels.formula.api as sm

lm = sm.ols('infant~ Accountability + aid + loggdp + gdp_grow + \
                        resourcesdep_hm + gini2 + lnpop + urban_cow + \
                        C(country_name) + C(year)  + C(violence_domestic) + C(communist) + \
                        + rx_infant + v2x_corr - 1', data).fit()
# print(lm.summary())
coefs = lm.params.to_dict()
year_effects = np.zeros((m,))
country_effects = np.zeros((n,))
violence_effects = np.zeros((11,))
communist_effects = np.zeros((2,))
for i in range(1,m):
    year_effects[i] = coefs['C(year)[T.' + str(years[i]) + ']']
for i in range(n):
    country_effects[i] = coefs['C(country_name)[' + countries[i] + ']']
for i in range(1,11):
    violence_effects[i] = coefs['C(violence_domestic)[T.' + str(i) + '.0]']
communist_effects[1] = coefs['C(communist)[T.1.0]']
noise_var = lm.scale
year_os = np.max(np.abs(year_effects))**2

covariate_names = ["Accountability", "aid", "loggdp","gdp_grow", \
                        "resourcesdep_hm","gini2","lnpop","urban_cow", \
                       "rx_infant", "v2x_corr"]
x_weights = list(map(coefs.get, covariate_names))
x_os = np.max(np.abs(list(map(lm.bse.to_dict().get, covariate_names))))**2


initialize model

In [39]:
likelihood = GaussianLikelihood()
likelihood.noise = 4 
# N = train_x.size()[0]
# inducing_points = train_x[torch.randperm(N)[:(N//20)]]
model = GPModel(train_x, train_y, likelihood, n)

# initialize model parameters
hypers = {
    'unit_mean_module.base_mean.constantvector': torch.tensor(country_effects),
    'unit_covar_module.base_kernel.kernels.0.lengthscale': torch.tensor(10),
    'unit_covar_module.base_kernel.kernels.1.lengthscale': torch.tensor(1),
    'unit_covar_module.outputscale': torch.tensor(year_os),
    'x_mean_module.base_mean.weights': torch.tensor(x_weights),
    'x_covar_module.outputscale': torch.tensor(x_os),
    'categorical_mean_module.0.base_mean.constantvector': torch.tensor(violence_effects),
    'categorical_mean_module.1.base_mean.constantvector': torch.tensor(communist_effects)
}    

model.initialize(**hypers)

GPModel(
  (likelihood): GaussianLikelihood(
    (noise_covar): HomoskedasticNoise(
      (raw_noise_constraint): GreaterThan(1.000E-04)
    )
  )
  (unit_mean_module): MaskMean(
    (base_mean): ConstantVectorMean()
  )
  (unit_covar_module): ScaleKernel(
    (base_kernel): ProductKernel(
      (kernels): ModuleList(
        (0): RBFKernel(
          (raw_lengthscale_constraint): Positive()
        )
        (1): RBFKernel(
          (raw_lengthscale_constraint): Positive()
        )
      )
    )
    (raw_outputscale_constraint): Positive()
  )
  (x_covar_module): ScaleKernel(
    (base_kernel): RBFKernel(
      (raw_lengthscale_constraint): Positive()
    )
    (raw_outputscale_constraint): Positive()
  )
  (x_mean_module): MaskMean(
    (base_mean): LinearMean()
  )
  (categorical_mean_module): ModuleList(
    (0): MaskMean(
      (base_mean): ConstantVectorMean()
    )
    (1): MaskMean(
      (base_mean): ConstantVectorMean()
    )
  )
)

train model by optimizing hypers

In [40]:
# train model
model.train()
likelihood.train()

torch.manual_seed(12345)

# freeze length scale in the country component in unit covar
# freeze constant unit means
all_params = set(model.parameters())
final_params = list(all_params - \
            {model.unit_covar_module.base_kernel.kernels[1].raw_lengthscale})

# optimizer = torch.optim.Adam([
#     {'params': final_params},
#     {'params': likelihood.parameters()},
# ], lr=0.1)
optimizer = torch.optim.Adam(final_params, lr=0.1)

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# loss_diff = 1
# last_loss = 200
# iter = 0
# while(loss_diff>=1e-2):
for iter in range(50):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(train_x)
    # Calc loss and backprop gradients
    loss = -mll(output, train_y)
    loss.backward()
    if iter % 5 == 0:
        print('Iter %d - Loss: %.3f '  % (
            iter, loss.item()
        ))
    # loss_diff = np.abs(last_loss-loss.item())
    # last_loss = loss.item()
    optimizer.step() 
    # iter += 1

torch.save(model.state_dict(), "AIM_GPR_model.pth")

Iter 0 - Loss: 5.814 
Iter 5 - Loss: 5.697 
Iter 10 - Loss: 5.149 
Iter 15 - Loss: 4.657 
Iter 20 - Loss: 4.273 
Iter 25 - Loss: 4.038 
Iter 30 - Loss: 3.833 
Iter 35 - Loss: 3.658 
Iter 40 - Loss: 3.536 
Iter 45 - Loss: 3.437 


generate posterior; compute RMSE

In [41]:
model.eval()
likelihood.eval()

with torch.no_grad(), gpytorch.settings.fast_pred_var():
    observed_pred = likelihood(model(train_x))

RMSE = np.square((observed_pred.mean - train_y).detach().numpy()).mean()**0.5
print(RMSE)

/Users/yahoo/anaconda3/lib/python3.7/site-packages/gpytorch/models/exact_gp.py:275: GPInputWarning: The input matches the stored training data. Did you forget to call model.train()?
  GPInputWarning,


2.3345288778754743


use autogradient to generate posterior variance of marginal effects by small batches

In [42]:
model.eval()
likelihood.eval()

df_std = np.zeros((train_x.size(0),train_x.size(1)))
x_grad = np.zeros((train_x.size(0),train_x.size(1)))
evidence_gp = 0
# small batches of size 100
for i in range(train_x.size(0)//100):
    with gpytorch.settings.fast_pred_var():
        test_x = train_x[(i*100):(i*100+100)].clone().detach().requires_grad_(True)
        observed_pred = likelihood(model(test_x))
        dydtest_x = torch.autograd.grad(observed_pred.mean.sum(), test_x, retain_graph=True)[0]
        x_grad[(i*100):(i*100+100)] = dydtest_x
        loss = mll(observed_pred, train_y[(i*100):(i*100+100)])
        evidence_gp += loss.item() * observed_pred.mean.size(0)

        n_samples = 100
        sampled_pred = observed_pred.rsample(torch.Size([n_samples]))
        sampled_dydtest_x = torch.stack([torch.autograd.grad(pred.sum(), test_x, retain_graph=True)[0] for pred in sampled_pred])
        df_std[(i*100):(i*100+100)] = sampled_dydtest_x.std(0)
        
# last 100 rows
with gpytorch.settings.fast_pred_var():
    test_x = train_x[(100*i+100):].clone().detach().requires_grad_(True)
    observed_pred = likelihood(model(test_x))
    dydtest_x = torch.autograd.grad(observed_pred.mean.sum(), test_x, retain_graph=True)[0]
    x_grad[(100*i+100):] = dydtest_x
    loss = mll(observed_pred, train_y[(100*i+100):])
    evidence_gp += loss.item() * observed_pred.mean.size(0)

    n_samples = 100
    sampled_pred = observed_pred.rsample(torch.Size([n_samples]))
    sampled_dydtest_x = torch.stack([torch.autograd.grad(pred.sum(), test_x, retain_graph=True)[0] for pred in sampled_pred])
    df_std[(100*i+100):] = sampled_dydtest_x.std(0)
    

accesse marginal effects of regressors including accountability

In [43]:
est_std = np.square(df_std).mean(axis=0).round(decimals=3)**0.5
est_std = (df_std).mean(axis=0).round(decimals=3)
results = pd.DataFrame({"x": covariate_names, \
                        'est_mean': x_grad.mean(axis=0)[2:12],
                        'est_std': est_std[2:12]})
results["t"] = results['est_mean'].values/results['est_std'].values
results["pvalue"] = norm.cdf(results["t"].values)
print(results)

                 x   est_mean  est_std           t        pvalue
0   Accountability  -4.292333    0.029 -148.011497  0.000000e+00
1              aid  -0.037023    0.076   -0.487144  3.130781e-01
2           loggdp -10.252744    1.955   -5.244370  7.840853e-08
3         gdp_grow   0.021200    0.246    0.086179  5.343378e-01
4  resourcesdep_hm   0.009873    0.096    0.102845  5.409572e-01
5            gini2  -0.001887    0.208   -0.009071  4.963813e-01
6            lnpop -17.964225    1.285  -13.979942  1.033286e-44
7        urban_cow  -0.075183    0.178   -0.422375  3.363757e-01
8        rx_infant   0.634666    0.487    1.303216  9.037495e-01
9         v2x_corr  -3.723241    1.052   -3.539202  2.006689e-04


compare model evidence and BIC

In [44]:
evidence_lm = lm.llf

num_params = 0
for p in final_params:
    num_params += p.reshape((-1,1)).size(0)
bic_gp = num_params*np.log(train_x.size(0)) + 2*loss.item()
bic_lm = lm.bic

print("lm evidence: {:.2f}\n".format(evidence_lm))
print("gpm evidence: {:.2f}\n".format(evidence_gp))
print("lm bic: {:.2f}\n".format(bic_lm))
print("gpm bic: {:.2f}\n".format(bic_gp))

lm evidence: -15435.67

gpm evidence: -10960.33

lm bic: 32664.42

gpm bic: 1536.82



In [47]:
print(model.unit_covar_module.base_kernel.kernels[0].lengthscale) # temporal cov ls
print(model.unit_covar_module.base_kernel.kernels[1].lengthscale) # spatial cov ls
print(model.unit_covar_module.outputscale) # spatial temporal cov os
print(model.x_mean_module.base_mean.weights) # covariate linear mean effect
print(model.x_covar_module.outputscale) # covariate kernel os
print(model.x_covar_module.base_kernel.lengthscale) # covariate kernel os

tensor([[6.3811]], grad_fn=<SoftplusBackward0>)
tensor([[1.]], grad_fn=<SoftplusBackward0>)
tensor(41.7539, grad_fn=<SoftplusBackward0>)
Parameter containing:
tensor([[-4.2979e+00],
        [-2.9468e-02],
        [-1.0524e+01],
        [ 6.7370e-03],
        [ 1.4616e-02],
        [ 7.7441e-03],
        [-1.7884e+01],
        [-7.9762e-02],
        [ 6.1208e-01],
        [-3.3652e+00]], requires_grad=True)
tensor(7.2707, grad_fn=<SoftplusBackward0>)
tensor([[3.9983, 5.0333, 0.4293, 5.0150, 4.9576, 4.8648, 0.3747, 4.8716, 4.8539,
         0.2553]], grad_fn=<SoftplusBackward0>)
